# Nursery Data Set

Nursery Data Set é um problema de classificação que pode ser encontrado no [UCA Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/nursery).

Este problema tem como contexto um modelo hierarquico para creches, desenvolvido para classificar o pedido de uma família. Seu objetivo é, dado os atributos da família, classificar
qual a prioridade da mesma para uma vaga na creche.

## 1. Dados

No dataset, há um total de 8 _features_ categóricas, onde cada uma delas têm valores qualitativos associados:

| Número da coluna   | _Feature_ | Descrição |Valores possíveis                                                        |
| :----------------: | :-------- | --------- |:----------------------------------------------------------------------- |
| 1                  | parents   | Ocupação dos pais              | usual, pretentious, great_pret                     |
| 2                  | has_nurs  | Creche atual                   | proper, less_proper, improper, critical, very_crit |
| 3                  | form      | Formato da família             | complete, completed, incomplete, foster            |
| 4                  | children  | Número de crianças             | 1, 2, 3, more                                      |
| 5                  | housing   | Condições de moradia           | convenient, less_conv, critical                    |
| 6                  | finance   | Situação financeira da família | convenient, inconv                                 |
| 7                  | social    | Condições sociais              | non-prob, slightly_prob, problematic               |
| 8                  | health    | Condições de saúde             | recommended, priority, not_recom                   |

Além das _features_, existe uma 9ª coluna _target_, que a denominamos de _classification_, onde os valores possíveis são: `not_recom, priority, recommended, spec_prior, very_recom`

## 2. Análise dos Dados

Os dados estão alocados no diretório `./data`, onde foi gerado um arquivo .csv partir do _dataset_ original, denomidado `nursey.data.csv`. Utilizamos várias bibliotecas para manipulação e visualização dos dados, onde podemos citar principalmente: pandas, numpy, matplotlib, pydot, seaborn, entre outras.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

A amostra que trabalhamos contém 12960 registros, onde, de acordo com o algorítmo utilizado, foi dividido em sub-amostras para treino e testes.

In [2]:
columns = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'classification']
df = pd.read_csv('./data/nursery.data.csv', header=None, names=columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12960 entries, 0 to 12959
Data columns (total 9 columns):
parents           12960 non-null object
has_nurs          12960 non-null object
form              12960 non-null object
children          12960 non-null object
housing           12960 non-null object
finance           12960 non-null object
social            12960 non-null object
health            12960 non-null object
classification    12960 non-null object
dtypes: object(9)
memory usage: 911.3+ KB


O _dataset_ não possui valores ausentes, o que facilitou o tratamento dos dados.

In [3]:
df.describe()

,parents,has_nurs,form,children,housing,finance,social,health,classification
count,12960,12960,12960,12960,12960,12960,12960,12960,12960
unique,3,5,4,4,3,2,3,3,5
top,great_pret,critical,complete,3,critical,inconv,slightly_prob,not_recom,not_recom
freq,4320,2592,3240,3240,4320,6480,4320,4320,4320


## 3. Tratamento dos Dados

### 3.1 _Feature Engineering_

Visto que todas as _features_ do _dataset_ são categóricas (não possuem valores quantitativos e não têm uma ordem lógica), foi necessário ajustar o dataset para transformá-las em variáveis _dummy_, para terem valores numéricos que fazem sentido no nosso contexto e poderem ser manipuladas.

Foi utilizado o método `get_dummies` do pandas para realizar o passo mencionado anteriormente.

In [5]:
X = pd.get_dummies(df.drop('classification', axis=1))
X.head()

,parents_great_pret,parents_pretentious,parents_usual,has_nurs_critical,has_nurs_improper,has_nurs_less_proper,has_nurs_proper,has_nurs_very_crit,form_complete,form_completed,...,housing_critical,housing_less_conv,finance_convenient,finance_inconv,social_nonprob,social_problematic,social_slightly_prob,health_not_recom,health_priority,health_recommended
0,0,0,1,0,0,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,1
1,0,0,1,0,0,0,1,0,1,0,...,0,0,1,0,1,0,0,0,1,0
2,0,0,1,0,0,0,1,0,1,0,...,0,0,1,0,1,0,0,1,0,0
3,0,0,1,0,0,0,1,0,1,0,...,0,0,1,0,0,0,1,0,0,1
4,0,0,1,0,0,0,1,0,1,0,...,0,0,1,0,0,0,1,0,1,0


Já para a columa _target_, transformamos seus valores categóricos em números correspondentes.

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['classification'])
y

array([2, 1, 0, ..., 3, 3, 0], dtype=int64)

Após isso, temos o _dataset_ totalmente numérico e pronto para ser nos modelos de aprendizagem. De 9 colunas (incluindo a _target_), o _dataset_ passou a ter 28 colunas.

In [7]:
numeric_y = pd.DataFrame(y, columns=['classification'])
numeric_y.head()
numeric_df = pd.concat([X, numeric_y], axis=1)
numeric_df.head()

,parents_great_pret,parents_pretentious,parents_usual,has_nurs_critical,has_nurs_improper,has_nurs_less_proper,has_nurs_proper,has_nurs_very_crit,form_complete,form_completed,...,housing_less_conv,finance_convenient,finance_inconv,social_nonprob,social_problematic,social_slightly_prob,health_not_recom,health_priority,health_recommended,classification
0,0,0,1,0,0,0,1,0,1,0,...,0,1,0,1,0,0,0,0,1,2
1,0,0,1,0,0,0,1,0,1,0,...,0,1,0,1,0,0,0,1,0,1
2,0,0,1,0,0,0,1,0,1,0,...,0,1,0,1,0,0,1,0,0,0
3,0,0,1,0,0,0,1,0,1,0,...,0,1,0,0,0,1,0,0,1,2
4,0,0,1,0,0,0,1,0,1,0,...,0,1,0,0,0,1,0,1,0,1


No fim, foi gerado um csv para utilizar nos modelos.

In [14]:
numeric_df.to_csv('numeric_nursery_data.csv', sep=',', encoding='utf-8')

## 4. Modelos de Aprendizagem

Dado o contexto do problema, foi feito uma pesquisa sobre os modelos que poderiam serem utilizados para resolver o mesmo. Os modelos selecionados foram escolhidos com base na leitura dos artigos relacionados e do [Machine Learning Map](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html).

<img height="auto" width="1000" src="./images/ml_map.png">

Com isso, os modelos selecionados foram:
- KNN
- SVM e Linear SVC
- Árvore de Decisão
- Naive Bayes

### 4.1 KNN

O algorítmo denominado de KNN vem da abreviação do nome _K nearest neighboors_, que significa o K vizinhos mais próximos. É um dos classificadores mais comums e utilizados em problemas de classificação. É bastante usado também por ter bons resultados e ser de fácil compreensão.

O KNN tem o objetivo de classificar o dado de uma amostra, baseado em amostras vizinhas. Em sua implementação matemática, dois pontos chaves são traçados, que são a distância e o valor de K. Para representar a distância, é comumente utilizada a Distância Euclidiana, mas existem outras formas de representá-la.

O valor de K pode ser variado, cabendo à quem implementa determinar o seu valor. É recomendado que o K seja um valor ímpar ou primo, mas é possível também que valores comuns sejam adequados pro seu problema. Para achar o melhor valor de K, existem várias técnicas para otimizar o seu algorítmo, mas essa procura pode tornar o desempenho do modelo mais lento. Ainda assim, o valor de K mais usado para problemas simples é através do método empírico.

Abaixo, segue a implementação do KNN com algumas técnicas diferentes.

### 4.2 SVM

SVM é a abreviação do termo em inglês _Support Vector Machines_, que é uma técnica de aprendizado de máquina supervisionado baseado em análise estatística, que é bastante utilizada para reconhecimento de padrões.

Um modelo SVM é uma representação de exemplos como pontos no espaço, mapeados de maneira que os exemplos de cada categoria sejam divididos por um espaço claro que seja tão amplo quanto possível. Os novos exemplos são então mapeados no mesmo espaço e preditos como pertencentes a uma categoria baseados em qual o lado do espaço eles são colocados.

O que uma SVM faz é encontrar uma linha de separação, mais comumente chamada de hiperplano entre dados de duas ou mais classes. Essa linha busca maximizar a distância entre os pontos mais próximos em relação a cada uma das classes.

### 4.3 Árvore de Decisão

### 4.4 Naive Bayes